In [2]:
import boto3
import os

#画像は全部で19049個

def get_s3_existing_keys(bucket_name, prefix):
    """S3バケット内の指定されたプレフィックスにある既存のキーを取得する"""
    session = boto3.Session(profile_name='personal')
    # S3クライアントを作成
    s3 = session.client('s3')
    existing_keys = set()
    continuation_token = None

    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        if 'Contents' not in response:
            break

        for obj in response['Contents']:
            # プレフィックス部分を除いたキー名を追加
            existing_keys.add(obj['Key'].replace(prefix + "/", ""))

        if response.get('IsTruncated'):  # 次のページがあるか
            continuation_token = response['NextContinuationToken']
        else:
            break

    return existing_keys

def upload_missing_files(bucket_name, prefix, local_folder):
    """ローカルフォルダ内のS3に存在しないファイルをアップロードする"""
    session = boto3.Session(profile_name='personal')
    # S3クライアントを作成
    s3 = session.client('s3')
    # S3上の既存のファイルリストを取得
    existing_keys = get_s3_existing_keys(bucket_name, prefix)
    print(f"Existing keys in S3: {existing_keys}")

    # ローカルフォルダ内のファイルリストを取得
    for root, _, files in os.walk(local_folder):
        for file in files:
            local_file_path = os.path.join(root, file)
            # S3キーはローカルパスからフォルダを除いた相対パス
            s3_key = os.path.relpath(local_file_path, local_folder).replace("\\", "/")  # Windows対策でパス区切りを変換

            if s3_key not in existing_keys:
                # S3キーにプレフィックスを付加
                full_s3_key = f"{prefix}/{s3_key}"
                print(f"Uploading {local_file_path} to {full_s3_key}")
                s3.upload_file(local_file_path, bucket_name, full_s3_key)
            else:
                print(f"Skipping {file} (already exists in S3)")

# 実行例
bucket_name = "dev-kusahata-2"
prefix = "test_photos"
local_folder = "/Users/jiangwenchu/Desktop/RIISE/webapp_checking/test_photos"

upload_missing_files(bucket_name, prefix, local_folder)


Existing keys in S3: {'', '365eff2f-5de5-4896-8824-ff0e017c05d0_1.jpg', '2ef0cd5a-2a80-4aa6-aab9-c088f8196639_2.jpg', '07cfccd6-5932-4f4f-9174-e76ef0346206_2.jpg', '41c9b957-e2c8-47cc-9982-9ef63fc7d7d7_2.jpg', '6224328c-fe08-4f22-bef8-b3e45ec329a0_2.jpg', '31a6c0b6-129c-4800-8ed6-808758770b6e_2.jpg', 'a4206480-8dc6-472c-b594-cb70a9eac75f_3.jpg', 'd247e78c-afab-4622-b823-e62bbdbcfde7_3.jpg', 'ee6706f8-bd68-4db6-8c32-3288ee2b4bc8_1.jpg', '153bf2f3-c128-48c8-b88f-3f1aca0f1893_1.jpg', 'd557b3ee-1b96-4c33-9d56-77dba1187d00_2.jpg', 'fbe302d5-fcbc-4f12-8a2b-6291d084f739_4.jpg', '98348454-5efe-401a-a6ba-f0ed3300ca1b_4.jpg', '530966f1-f5c2-4a32-8525-cb34b853b349_1.jpg', 'f952afe3-4f0c-4389-8606-81baffdb457f_3.jpg', '4661b1f4-03fa-4eca-886e-b54fbc468fed_2.jpg', '31dac12b-fcde-4eff-8e6b-c92bd1a63f6b_2.jpg', 'fcac54f8-25b6-4e5e-8afe-294c66aa2bcb_6.jpg', '1b2bddef-6a20-4bcf-9891-b0b63bf9b170_3.jpg', '5558f3cc-2b1e-42e3-b431-b6adba958b77_4.jpg', '3444051b-7d0f-4747-ae6e-af6edf870182_2.jpg', 'd795b51